# Package Functionalities Demo:

## Preparation: load data, functions, libraries of dependency...

In [1]:
pwd()

"/Users/zifanyu/Documents/GitHub/BulkLMM.jl"

In [2]:
cd("test")

In [3]:
using BenchmarkTools
using LoopVectorization

In [78]:
include("BXDdata_for_test.jl");
include("../src/transform_helpers.jl");
include("../src/scan.jl");
include("../src/bulkscan.jl");
include("testHelpers.jl");

In [25]:
traitID = 7919;
pheno_y = reshape(pheno[:, traitID], :, 1);

## 1. Simple Univariate Scans:

Given the raw phenotype info of one trait, the genotype info and the corresponding pre-calculated kinship matrix, return the lod scores of all genetic markers on that particular trait.

In [26]:
?scan

search: scan scan_alt scan_null scan_lite scan_perms scan_perms_lite



```
scan(y, g, K, reml, method)
```

Performs genome scan for univariate trait and each of the gene markers, one marker at a time (one-df test) 

# Arguments

  * y = 1d array of floats consisting of the N observations for a certain quantitative trait (dimension: N*1)
  * g = 2d array of floats consisting of all p gene markers (dimension: N*p)
  * K = 2d array of floats consisting of the genetic relatedness of the N observations (dimension:N*N)
  * prior_a = a float of prior distribution parameter
  * prior_b = a float of prior distribution parameter

# Keyword arguments

  * addIntercept = Boolean flag indicating if an intercept column needs to be added to the design matrix; Default is to add an intercept
  * reml = Bool flag indicating if VCs are estimated by REML likelihood; Default is ML
  * assumption = String indicating whether the variance component parameters are the same across all markers (null) or not (alt); Default is `null`
  * method = String indicating the matrix factorization method to use; Default is QR.

# Value

A list of output values are returned:

  * out00.sigma2 = Float; estimated marginal variance due to random errors (by null lmm)
  * out00.h2 = Float; estimated heritability (by null lmm)
  * lod = 1d array of floats consisting of the lod scores of this trait and all markers (dimension: p*1)

# Some notes

```
This function calls either `scan_null` or `scan_alt` depending on the input passed as `method`.
Output data structure might need some revisions.
```


### Assumption (1): VCs are estimated once from the null model

In [27]:
?scan_null

search: scan_null



```
scan_null(y, g, K, reml, method)
```

Performs genome scan for univariate trait and each of the gene markers, one marker at a time,  assuming the variance components are the same for all markers.

# Arguments

  * y = 1d array of floats consisting of the N observations for a certain quantitative trait (dimension: N*1)
  * g = 2d array of floats consisting of all p gene markers (dimension: N*p)
  * K = 2d array of floats consisting of the genetic relatedness of the N observations (dimension:N*N)

# Keyword arguments

  * addIntercept = Boolean flag indicating if an intercept column needs to be added to the design matrix; Default is to add an intercept
  * reml = Bool flag indicating if VCs are estimated by REML likelihood; Default is ML
  * method = String indicating the matrix factorization method to use; Default is QR.

# Value

A list of output values are returned:

  * out00.sigma2 = Float; estimated marginal variance due to random errors (by null lmm)
  * out00.h2 = Float; estimated heritability (by null lmm)
  * lod = 1d array of floats consisting of the lod scores of this trait and all markers (dimension: p*1)

# Some notes

```
This is a subsequent function that does univariate scan. The variance components are estimated once 
and used for all markers. To be called by the `scan` function when the `method = ` field is passed 
as `null` (default).
```


In [79]:
@time test_null = scan(pheno_y, geno, kinship; addIntercept = true, reml = false, assumption = "null", method = "cholesky");

  0.022348 seconds (73.58 k allocations: 33.192 MiB)


In [80]:
test_null.sigma2_e # sigma2_e

0.04983520992588347

In [82]:
test_null.h2_null

0.8960637228416714

In [83]:
test_null.lod

7321-element Vector{Float64}:
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360229954236
 0.008195284184264295
 0.00987347031983854
 ⋮
 0.09285575916070832
 0.09285575883025671
 0.09285575883025671
 0.011173043848082764
 0.008215696058314959
 0.008215696058314959
 0.01502332773404691
 0.02895771332730057
 0.02731129976924651
 0.04498777747884275
 0.012828276502853786
 0.012828276502853786

### Assumption (2): VCs are re-estimated everytime for each marker scan

In [32]:
?scan_alt

search: scan_alt scan_lite_univar



```
scan_alt(y, g, K, reml)
```

Performs genome scan for univariate trait and each of the gene markers, one marker at a time (one-df test), assuming the variance components may not be the same for all markers. 

# Arguments

  * y = 1d array of floats consisting of the N observations for a certain quantitative trait (dimension: N*1)
  * g = 2d array of floats consisting of all p gene markers (dimension: N*p)
  * K = 2d array of floats consisting of the genetic relatedness of the N observations (dimension:N*N)

# Keyword arguments

  * addIntercept = Boolean flag indicating if an intercept column needs to be added to the design matrix; Default is to add an intercept
  * reml = Bool flag indicating if VCs are estimated by REML likelihood; Default is ML
  * method = String indicating the matrix factorization method to use; Default is QR.

# Value

A list of output values are returned:

  * out00.sigma2 = Float; estimated marginal variance due to random errors (by null lmm)
  * out00.h2 = Float; estimated heritability (by null lmm)
  * lod = 1d array of floats consisting of the lod scores of this trait and all markers (dimension: p*1)

# Some notes

```
This is a subsequent function that does univariate scan. For every scan for each genetic marker, the 
variance components will need to be re-estimated. To be called by the `scan` function when the `method = ` 
field is passed as `alt`.
```


In [85]:
@time test_alt = scan(pheno_y, geno, kinship; addIntercept = true, reml = false, assumption = "alt", method = "cholesky");

  0.639650 seconds (2.32 M allocations: 878.476 MiB, 23.62% gc time)


In [86]:
test_alt.sigma2_e 

0.04983520992588347

In [87]:
test_alt.h2_each_marker

7321-element Vector{Float64}:
 0.8959152120195115
 0.8959152120195115
 0.8959152120195115
 0.8959152120195115
 0.8959152120195115
 0.8959152120196866
 0.8959152120195115
 0.8959152120195115
 0.8959152120195115
 0.8959152120195115
 0.8959152121140267
 0.8959152374339192
 0.8957761212977645
 ⋮
 0.9023536181416946
 0.9023536181056799
 0.9023536181056799
 0.8984874509594057
 0.8951424229522367
 0.8951424229522367
 0.8948313365595942
 0.8942228015686338
 0.8942884362217399
 0.893637575195784
 0.894429331508082
 0.894429331508082

In [88]:
test_alt.lod

7321-element Vector{Float64}:
 0.008197761427624278
 0.008197761427624278
 0.008197761427624278
 0.008197761427624278
 0.008197761427624278
 0.008197761427624278
 0.008197761427624278
 0.008197761427624278
 0.008197761427624278
 0.008197761427624278
 0.008197761328216775
 0.00819668480291893
 0.009878635627355274
 ⋮
 0.09535348758648882
 0.09535348724995858
 0.09535348724995858
 0.011536179594525912
 0.008268469217227748
 0.008268469217227748
 0.015117504699342935
 0.029166995976271558
 0.02750600122533986
 0.04535019721089667
 0.012992620923693021
 0.012992620923693021

In [90]:
summary_tab1 = DataFrame(Assumption = ["Null", "Alt"], 
                         Sigma2_e = [test_null.sigma2_e, test_alt.sigma2_e], 
                         H2 = [test_null.h2_null, test_alt.h2_null]);
print(summary_tab1)

2×3 DataFrame
 Row │ Assumption  Sigma2_e   H2       
     │ String      Float64    Float64  
─────┼─────────────────────────────────
   1 │ Null        0.0498352  0.896064
   2 │ Alt         0.0498352  0.896064

In [91]:
summary_tab1_lods = DataFrame(Marker = collect(1:10), Null = test_null.lod[1:10], Alt = test_alt.lod[1:10]);
print(summary_tab1_lods)

10×3 DataFrame
 Row │ Marker  Null        Alt        
     │ Int64   Float64     Float64    
─────┼────────────────────────────────
   1 │      1  0.00819636  0.00819776
   2 │      2  0.00819636  0.00819776
   3 │      3  0.00819636  0.00819776
   4 │      4  0.00819636  0.00819776
   5 │      5  0.00819636  0.00819776
   6 │      6  0.00819636  0.00819776
   7 │      7  0.00819636  0.00819776
   8 │      8  0.00819636  0.00819776
   9 │      9  0.00819636  0.00819776
  10 │     10  0.00819636  0.00819776

In [93]:
mean(test_null.lod .< test_alt.lod)

1.0

## 2. Univariate Scans with permutation tests:

In [94]:
@time test_perms = scan_perms(pheno_y, geno, kinship; 
                              prior_variance = 1.0, prior_sample_size = 0.1,
                              addIntercept = true, reml = false, method = "cholesky",
                              nperms = 1000, original = true);

  6.711660 seconds (103.90 k allocations: 17.629 GiB, 42.29% gc time)


In [95]:
size(test_perms)

(1001, 7321)

In [96]:
@time test_perms_lite = scan_perms_lite(pheno_y, geno, kinship; 
                                   prior_variance = 1.0, prior_sample_size = 0.1,
                                   addIntercept = true, reml = false, method = "cholesky",
                                   nperms = 1000, original = true);
test_perms_lite = permutedims(test_perms_lite);

  0.234718 seconds (178.26 k allocations: 99.685 MiB, 52.46% compilation time)


In [97]:
sumSqDiff(test_perms, test_perms_lite)

8.861391305439763e-22

In [98]:
@time test_null_corrected = scan(pheno_y, geno, kinship; prior_variance = var(pheno_y), prior_sample_size = 0.1, method = "cholesky");

  0.026852 seconds (82.63 k allocations: 33.731 MiB, 21.78% compilation time)


In [100]:
sum((test_null_corrected.lod .- test_perms_lite[1, :]).^2)

4.352772803058051e-18

In [102]:
hcat(test_perms_lite[1, :], test_null_corrected.lod)

7321×2 Matrix{Float64}:
 0.00830149  0.00830149
 0.00830149  0.00830149
 0.00830149  0.00830149
 0.00830149  0.00830149
 0.00830149  0.00830149
 0.00830149  0.00830149
 0.00830149  0.00830149
 0.00830149  0.00830149
 0.00830149  0.00830149
 0.00830149  0.00830149
 0.00830149  0.00830149
 0.00830039  0.00830039
 0.0101448   0.0101448
 ⋮           
 0.0876171   0.0876171
 0.0876171   0.0876171
 0.0876171   0.0876171
 0.00923218  0.00923218
 0.00903044  0.00903044
 0.00903044  0.00903044
 0.0161049   0.0161049
 0.0305571   0.0305571
 0.028855    0.028855
 0.0470821   0.0470821
 0.0142602   0.0142602
 0.0142602   0.0142602